# DS Primer - Week 2 - Data Management

## Socio-Political Analysis on Human Development Index

### 1. Importing and Loading File - Country Events Dataset

In order to get the various armed conflict events happenning in a country, we need to find a source where we can find the events in countries in addition to the particular year on which an event occurs.
The Uppasala Conflict Data Program dashboard would be a great source for this data - https://ucdp.uu.se/

We have already consolidated and downloaded a dataset from this datasource and you can find it in `Datasets/Dataset_Country_Events.csv` path

In [137]:
import pandas as pd
import numpy as np

country_events = pd.read_csv('Datasets/Dataset_Country_Events.csv')

In [144]:
country_events.head()

,year,active_year,type_of_violence,conflict_new_id,conflict_name,dyad_new_id,dyad_name,side_a_new_id,gwnoa,side_a,...,date_prec,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best,low,high
0,2010,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),...,1,2010-09-25,2010-09-25,2,0,0,0,2,2,2
1,2011,1,3,715,Government of Yemen (North Yemen) - Civilians,1182,Government of Yemen (North Yemen) - Civilians,123,678.0,Government of Yemen (North Yemen),...,1,2011-02-19,2011-02-19,0,0,0,0,0,0,2
2,2011,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),...,1,2011-04-16,2011-04-16,0,0,0,0,0,0,1
3,2012,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),...,1,2012-06-04,2012-06-04,4,1,0,0,5,5,5
4,2012,1,1,230,Yemen (North Yemen):Government,459,Government of Yemen (North Yemen) - AQAP,123,678.0,Government of Yemen (North Yemen),...,1,2012-12-09,2012-12-09,0,4,1,0,5,5,5


##### Data Granularity 

Here, it seems that each row represent a particular event that occured in a country

### 2. Data Preparation

In [139]:
country_events = country_events.drop(['id'], axis=1)
country_events = country_events[country_events.year.isin(['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015'])]

In [142]:
# iterrows() function is used for iterating through the rows in pandas
# strip() function is used to trim the string
# we are trimming the extra spaces in country column
for index, row in country_events.iterrows():
    row['country'] = (row['country']).strip()
    row['year'] = (row['country']).strip()      

#### 2.1 Data Normalization

We are planning to integrate the socio-politcal and the human development index data. For that you need to normalize the data so that you can merge the data across multiple data sources

In [143]:
# While looking at the country column it seems the names are more elaborate and would be difficult to join the data based on countries with other datasets
# So changing the name of the country names to a standard format
country_events.loc[country_events['country'] == 'Yemen (North Yemen)', 'country'] = 'Yemen'
country_events.loc[country_events['country'] == 'DR Congo (Zaire)', 'country'] = 'DR Congo'
country_events.loc[country_events['country'] == 'Madagascar (Malagasy)', 'country'] = 'Madagascar'
country_events.loc[country_events['country'] == 'Zimbabwe (Rhodesia)', 'country'] = 'Zimbabwe'
country_events.loc[country_events['country'] == 'Serbia (Yugoslavia)', 'country'] = 'Serbia'
country_events.loc[country_events['country'] == 'Cambodia (Kampuchea)', 'country'] = 'Cambodia'
country_events.loc[country_events['country'] == 'Russia (Soviet Union)', 'country'] = 'Russia'
country_events.loc[country_events['country'] == 'Myanmar (Burma)', 'country'] = 'Myanmar'

In [79]:
# For a common and uniform format let's convert the datatype of the column year from integer to a string
country_events['year'] = country_events['year'].apply(str)
country_events['country'] = country_events['country'].apply(str)

# Exporting the dataframe into a csv
country_events.to_csv('news_article_conflicts.csv')

#### 2.2 Data Aggregation

The data granularity of the dataset is every individual event that had happened in a country. 
But for a master dataset we might need aggregated count for a particular country

In [145]:
conflict_aggregate = country_events.groupby(['country','year'], as_index=False).agg({'best': 'sum', 'deaths_civilians': 'sum'})
conflict_aggregate

,country,year,best,deaths_civilians
0,Afghanistan,2005,1711,200
1,Afghanistan,2006,4982,477
2,Afghanistan,2007,7020,669
3,Afghanistan,2008,5660,763
4,Afghanistan,2009,6499,1099
...,...,...,...,...
507,Zimbabwe,2007,0,0
508,Zimbabwe,2008,253,253
509,Zimbabwe,2009,0,0
510,Zimbabwe,2011,1,1


In [146]:
# Renaming the columns
conflict_aggregate=conflict_aggregate.rename(columns = {'best':'total_deaths'})
conflict_aggregate=conflict_aggregate.rename(columns = {'deaths_civilians':'civilian_deaths'})

In [147]:
conflict_aggregate

,country,year,total_deaths,civilian_deaths
0,Afghanistan,2005,1711,200
1,Afghanistan,2006,4982,477
2,Afghanistan,2007,7020,669
3,Afghanistan,2008,5660,763
4,Afghanistan,2009,6499,1099
...,...,...,...,...
507,Zimbabwe,2007,0,0
508,Zimbabwe,2008,253,253
509,Zimbabwe,2009,0,0
510,Zimbabwe,2011,1,1


In [155]:
# Saving the aggregated file as csv
conflict_aggregate.to_csv('conflict_aggregate.csv')

### 2. Importing and Loading File - Human Development Index Data

The data source for this particular file can be found here -http://hdr.undp.org/en/content/human-development-index-hdi
The explanation of HDI will also be available here

In [156]:
#Forming the HDI data
import pandas as pd
hdi_data_payload = pd.read_csv('Datasets/Dataset_HDI_Data.csv',encoding = "ISO-8859-1")
hdi_data = hdi_data_payload[['Country','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']]

In [157]:
hdi_data.head()

,Country,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afghanistan,0.405,0.415,0.433,0.434,0.448,0.454,0.463,0.470,0.476,0.479,0.479
1,Albania,0.696,0.703,0.713,0.721,0.725,0.738,0.752,0.759,0.761,0.762,0.764
2,Algeria,0.686,0.690,0.697,0.705,0.714,0.724,0.732,0.737,0.741,0.743,0.745
3,Andorra,NaN,NaN,NaN,NaN,NaN,0.819,0.819,0.843,0.850,0.857,0.858
4,Angola,0.439,0.454,0.468,0.480,0.488,0.495,0.508,0.523,0.527,0.531,0.533


#### 2.1 Data Manipulation 
One of the important steps and needed steps in the industry

In [158]:
# here we will invert the axes and manipulate data such that country and year become a row value
hdi_data = hdi_data.melt(id_vars=['Country'])
hdi_data

,Country,variable,value
0,Afghanistan,2005,0.405
1,Albania,2005,0.696
2,Algeria,2005,0.686
3,Andorra,2005,NaN
4,Angola,2005,0.439
...,...,...,...
2063,Venezuela (Bolivarian Republic of),2015,0.767
2064,Viet Nam,2015,0.683
2065,Yemen,2015,0.482
2066,Zambia,2015,0.579


In [159]:
# Renaming and Cleaning the dataset
hdi_data=hdi_data.rename(columns = {'Country':'country'})
hdi_data=hdi_data.rename(columns = {'variable':'year'})

for index, row in hdi_data.iterrows():
    row['country'] = (row['country']).strip()
    row['year'] = (row['year']).strip()

#### 2. Data Normalization

In [160]:
hdi_data.loc[hdi_data['country'] == 'Bolivia (Plurinational State of)', 'country'] = 'Bolivia'
hdi_data.loc[hdi_data['country'] == 'Bosnia and Herzegovina', 'country'] = 'Bosnia-Herzegovina'
hdi_data.loc[hdi_data['country'] == 'Congo (Democratic Republic of the)', 'country'] = 'DR Congo'
hdi_data.loc[hdi_data['country'] == 'Côte d\'Ivoire', 'country'] = 'Ivory Coast'
hdi_data.loc[hdi_data['country'] == 'Iran (Islamic Republic of)', 'country'] = 'Iran'
hdi_data.loc[hdi_data['country'] == 'Lao People\'s Democratic Republic', 'country'] = 'Laos'
hdi_data.loc[hdi_data['country'] == 'Moldova (Republic of)', 'country'] = 'Moldova'
hdi_data.loc[hdi_data['country'] == 'Venezuela (Bolivarian Republic of)', 'country'] = 'Venezuela'
hdi_data.loc[hdi_data['country'] == 'Tanzania (United Republic of)', 'country'] = 'Tanzania'
hdi_data.loc[hdi_data['country'] == 'The former Yugoslav Republic of Macedonia', 'country'] = 'Macedonia'
hdi_data.loc[hdi_data['country'] == 'Viet Nam', 'country'] = 'Vietnam'

In [161]:
hdi_data.to_csv('hdi_master.csv')

#### 3. Data Merging

In [162]:
hdi_data

,country,year,value
0,Afghanistan,2005,0.405
1,Albania,2005,0.696
2,Algeria,2005,0.686
3,Andorra,2005,NaN
4,Angola,2005,0.439
...,...,...,...
2063,Venezuela,2015,0.767
2064,Vietnam,2015,0.683
2065,Yemen,2015,0.482
2066,Zambia,2015,0.579


In [164]:
print(type(conflict_aggregate.country[0]))
print(type(conflict_aggregate.year[0]))
print(type(hdi_data.country[0]))
print(type(hdi_data.year[0]))

<class 'str'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>


In [165]:
conflict_aggregate['year'] = conflict_aggregate.year.astype(str)

In [168]:
a = 2

In [170]:
str(a)

'2'

In [ ]:
country_events['year'] = country_events.year.apply(str)

In [166]:
# We now try to merge the country events data with the HDI data
master_data = pd.merge(conflict_aggregate, hdi_data,how="left", on=['country', 'year'])
master_data.to_csv('conflict_hdi.csv')

In [167]:
master_data

,country,year,total_deaths,civilian_deaths,value
0,Afghanistan,2005,1711,200,0.405
1,Afghanistan,2006,4982,477,0.415
2,Afghanistan,2007,7020,669,0.433
3,Afghanistan,2008,5660,763,0.434
4,Afghanistan,2009,6499,1099,0.448
...,...,...,...,...,...
507,Zimbabwe,2007,0,0,0.421
508,Zimbabwe,2008,253,253,0.419
509,Zimbabwe,2009,0,0,0.436
510,Zimbabwe,2011,1,1,0.464
